In [ ]:
# GPU 할당 코드
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6000)])
    except RuntimeError as e:
        print(e)
        

In [ ]:
# DenseNet169을 이용한 모델 생성

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.callbacks import ModelCheckpoint

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Train 데이터, Validation 데이터 위치 주소 
train_dir = '/home/workspace/user-workspace/crack_data_full_8_2/train'
val_dir = '/home/workspace/user-workspace/crack_data_full_8_2/validation'

# Train 데이터 생성 및 Augmentation
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=25,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)

validation_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
    train_dir,                      
    classes=['0','1'],               
    target_size = (136,136),
    batch_size = 30 ,           
    class_mode = 'binary',
   
                                                   )
validation_generator = validation_datagen.flow_from_directory(
    val_dir,           
    classes=['0','1'],  
    target_size = (136,136),  
    batch_size = 32 ,           
    class_mode = 'binary',
    
                                                   )

## Pretrained Network DenseNet169 사용
with tf.device('/device:GPU:0'):
    model_base = DenseNet169(weights='imagenet',
                             include_top=False,
                             input_shape=(136,136,3))

    model_base.trainable = False  

    model = Sequential()

    model.add(model_base)

    model.add(Flatten(input_shape=(4*4*1664,)))

    model.add(Dense(3600,
                    activation='relu'))
    model.add(BatchNormalization())
 
    model.add(Dense(1,
                    activation='sigmoid'))

    model.summary()
    
    lr = 2e-6
    ep = 50
    
    model.compile(optimizer=Adam(learning_rate = lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    best_weights_file="weights.best.hdf5"
    checkpoint = ModelCheckpoint(best_weights_file, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

    
    history = model.fit(train_generator,
                        steps_per_epoch=375,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=32,
                        verbose=1)
    
    
    model_base.trainable = False
    fine_tune = 35

    # 마지막 35개의 layer 재학습 
    for layer in model_base.layers[-fine_tune:]:
          layer.trainable = True

   
    model.compile(optimizer=Adam(learning_rate = lr),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # 재학습 진행
    history = model.fit(train_generator,
                             callbacks=[checkpoint],
                             steps_per_epoch=375,
                             epochs=ep,
                             validation_data=validation_generator,
                             validation_steps=32,
                             verbose=1)

    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.plot(train_acc, 'bo', color='r', label='training accuracy')
    plt.plot(val_acc, 'b', color='b', label='validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

    plt.plot(train_loss, 'bo', color='r', label='training loss')
    plt.plot(val_loss, 'b', color='b', label='validation loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()
    
    # model 저장위치 주소 입력 후 저장
    model.save('/user-workspace/model/3. model.h5/test_cnn_model_DenseNet169(136,136)_v3.h5')